In [13]:
import argparse
import tensorflow as tf
import glovedata
import tensorglove
from glovedata import FEATURES
import tensorglove_osc_server




In [43]:
train_steps = 2000
batch_size = 100
run_server = True

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=2000, type=int,
                    help='number of training steps')
parser.add_argument('--run_server', default=True, help='whether to run prediction server')

_StoreAction(option_strings=['--run_server'], dest='run_server', nargs=None, const=None, default=True, type=None, choices=None, help='whether to run prediction server', metavar=None)

In [44]:
def main(argv):
    """
    Run the training and serving.
    :param argv: Command line arguments.
    :return:
    """
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = glovedata.load_data()

    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    hidden_units = [12,12]
    model_dir = "model_{0}_{1}".format(hidden_units[0],hidden_units[1])

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=hidden_units,
        # The model must choose between 4 classes.
        n_classes=4,
        model_dir=model_dir)




    #for dataset in blah: for text_x and test_y

    model_dir="model_{0}_{1}".format(hidden_units[0], hidden_units[1])

    # Train the Model.
    classifier.train(
        input_fn=lambda: glovedata.train_input_fn(train_x, train_y,
                                                  batch_size),
        steps=train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda: glovedata.eval_input_fn(test_x, test_y,
                                                 batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # perform a sample prediction

    # Generate predictions from the model
    expected = ['Fist']
    feature_values = [0, 0, 0, 1, 0.2298394, 0.9228836, -0.1887825, -0.2445833, 0.03349165, -0.2397906, -0.1800468,
                      0.9533949, -0.07355202, 0.2745002, -0.2481479, 0.9261007, -4.470348E-08, 0.2213377, 1.490116E-08,
                      0.9751973, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, 0.09148686, -0.09309785, -0.6949129,
                      0.7071485, 7.105427E-15, 7.45058E-08, -0.7191889, 0.6948146, 1.490116E-08, -1.490116E-08,
                      -0.566272, 0.8242185, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -3.725291E-08, 5.960464E-08,
                      -0.7009093, 0.7132504, 1.490116E-08, 3.352762E-08, -0.772563, 0.6349381, -7.450572E-09,
                      2.980232E-08, -0.6148145, 0.7886717, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -0.06108833,
                      0.06216392, -0.6982422, 0.7105362, -3.352762E-08, 1.490116E-08, -0.7539417, 0.6569413,
                      5.960465E-08, 5.960463E-08, -0.5976215, 0.8017784, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1,
                      -0.1217117, 0.1238547, -0.6902609, 0.7024146, -5.029142E-08, 3.725291E-08, -0.819152, 0.5735765,
                      5.215406E-08, 5.960463E-08, -0.6427875, 0.7660446]
    # the data is expected to be a list of feature values (as it is configured for batching)
    predict_x = dict(zip(FEATURES, [[x] for x in feature_values]))

    predictions = classifier.predict(
        input_fn=lambda: glovedata.eval_input_fn(predict_x,
                                                 labels=None,
                                                 batch_size=batch_size))

    template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(glovedata.GESTURES[class_id],
                              100 * probability, expec))

    export_dir = classifier.export_savedmodel(
        export_dir_base="model_export",
        serving_input_receiver_fn=serving_input_receiver_fn)
    print('Exported to:', export_dir)

    if run_server:
        run_server(classifier)

In [45]:
#tf.logging.set_verbosity(tf.logging.INFO)
#tf.app.run(main)
main([])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_12_12', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c2ca93ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


/Users/joaomorado/Desktop/oxford_work/software_devolpment/hackathon/tensorglove/python/glovedata.py:54: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  data.convert_objects()
/Users/joaomorado/Desktop/oxford_work/software_devolpment/hackathon/tensorglove/python/glovedata.py:59: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  train_y.convert_objects()
/Users/joaomorado/Desktop/oxford_work/software_devolpment/hackathon/tensorglove/python/glovedata.py:61: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type spec

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_12_12/model.ckpt-4000


InvalidArgumentError: tensor_name = dnn/hiddenlayer_1/bias; shape in shape_and_slice spec [12] does not match the shape stored in checkpoint: [10]
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File 